In [ ]:
!pip install -q fastai torchvision --upgrade


# Download KMNIST via torchvision (if using PyTorch)
from torchvision import datasets
datasets.KMNIST(root='/content/data', train=True, download=True)
datasets.KMNIST(root='/content/data', train=False, download=True)

Dataset KMNIST
    Number of datapoints: 10000
    Root location: /content/data
    Split: Test

In [ ]:
!pip install -q keras

In [ ]:
!pip install -q tensorflow

In [ ]:

from fastai.learner import Learner
from fastai.metrics import accuracy
from IPython import get_ipython
from IPython.display import display
from torchvision.models.resnet import resnet18, ResNet, BasicBlock
from torchvision.datasets.mnist import MNIST
from fastai.vision import *
from fastai import *
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import BatchSampler, DataLoader
from fastai.data.all import DataLoaders
import torchvision.models as models


In [ ]:
# === Embedded KujuMNIST Dataset Class ===
import os
import numpy as np
from torch.utils.data import Dataset
from torchvision.datasets.utils import download_url
from pathlib import Path

class KujuMNIST_DS(Dataset):
    urls = [
        'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
        'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
        'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
        'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz',
    ]
    base_filename = 'kmnist-{}-{}.npz'
    data_filepart = 'imgs'
    labels_filepart = 'labels'

    def __init__(self, folder, train_or_test='train', download=False, num_classes=10, max_items=None, tfms=None):
        self.root = os.path.expanduser(folder)
        if download:
            self.download()

        self.train = (train_or_test == 'train')

        self.data = np.load(os.path.join(self.root, self.base_filename.format(train_or_test, self.data_filepart)))
        self.data = self.data['arr_0']
        self.targets = np.load(os.path.join(self.root, self.base_filename.format(train_or_test, self.labels_filepart)))
        self.targets = self.targets['arr_0']
        self.c = num_classes
        self.max_items = max_items
        self.tfms = tfms

    def __getitem__(self, index):
        cur_data = np.expand_dims(self.data[index], axis=-1)
        if self.tfms:
            cur_data = self.tfms(cur_data)
        target = int(self.targets[index])
        return cur_data, target

    def __len__(self):
        return self.max_items if self.max_items else len(self.data)

    def download(self):
        Path(self.root).mkdir(parents=True, exist_ok=True)
        for url in self.urls:
            filename = url.rpartition('/')[-1]
            file_path = os.path.join(self.root, filename)
            download_url(url, root=self.root, filename=filename, md5=None)


Calculate mean and std of the dataset for normalization

In [ ]:
trn_data = np.load('/content/kmnist-train-imgs.npz')
trn_data = trn_data['arr_0'] / 255
data_mean = trn_data.mean()
data_std = trn_data.std()
print(f'Mean: {data_mean}')
print(f'Std: {data_std}')

Mean: 0.1917621473589439
Std: 0.34834283034636876


## Prepare Datasets, DataLoaders and DataBunch

Optional: A random transformations on the images in training

In [ ]:
default_device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

transform_train = transforms.Compose(
    [transforms.ToPILImage(),
     #transforms.RandomAffine(degrees=7, translate=(0.1, 0.1), scale=(0.95, 1.05)),
     transforms.ToTensor(),
     transforms.Normalize((data_mean,), (data_std,)),
    ])

transform_valid = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.ToTensor(),
     transforms.Normalize((data_mean,), (data_std,)),
    ])

ROOT_FOLDER = '/content/'

trn_ds = KujuMNIST_DS(ROOT_FOLDER, train_or_test='train', download=False, tfms=transform_train)
val_ds = KujuMNIST_DS(ROOT_FOLDER, train_or_test='test', download=False, tfms=transform_valid)

trn_dl = DataLoader(trn_ds, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)
#databunch = DataBunch(path=ROOT_FOLDER, train_dl=trn_dl, valid_dl=val_dl, device=default_device)
databunch = DataLoaders(trn_dl, val_dl, device=default_device)

In [ ]:
# Get one batch from the training DataLoader
one_batch = next(iter(databunch.train))

# The batch is a tuple: (images, labels)
images, labels = one_batch

# Get the shape of the images tensor
image_shape = images.shape

# The number of channels is the second dimension (index 1) in the shape tuple
num_channels = image_shape[1]

print(f"Shape of one image batch: {image_shape}")
print(f"Number of input channels: {num_channels}")

Shape of one image batch: torch.Size([128, 1, 28, 28])
Number of input channels: 1


MNIST Dataset

In [ ]:
# dataset_transform = transforms.Compose([
#                transforms.ToTensor(),
#                transforms.Normalize((0.1307,), (0.3081,))
#            ])

# mnist_trn_ds = MNIST('/content/', train=True, download=False, transform=dataset_transform)
# mnist_val_ds = MNIST('/content/', train=False, download=False, transform=dataset_transform)

## VGG Model

Based on - https://github.com/kkweon/mnist-competition

In [ ]:
import math
class VGG(nn.Module):
    """
    Based on - https://github.com/kkweon/mnist-competition
    """
    def two_conv_pool(self, in_channels, f1, f2):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s

    def three_conv_pool(self,in_channels, f1, f2, f3):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.Conv2d(f2, f3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s


    def __init__(self, num_classes=10):
        super(VGG, self).__init__()
        self.l1 = self.two_conv_pool(1, 64, 64)
        self.l2 = self.two_conv_pool(64, 128, 128)
        self.l3 = self.three_conv_pool(128, 256, 256, 256)
        self.l4 = self.three_conv_pool(256, 256, 256, 256)

        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)



In [ ]:
vgg_model_with_norm = models.vgg16_bn(pretrained=True)
vgg_model_with_norm.classifier[6] = nn.Linear(4096, 10)
learn = Learner(databunch, VGG(), metrics=accuracy, loss_func = nn.CrossEntropyLoss(reduction='mean'))
# learn.load('vgg_model_with_norm')
# print('Model was loaded')

In [ ]:
learn.fit(1)

[0, 0.08840512484312057, 0.25744563341140747, 0.921999990940094, '00:18']


## ResNet Model

Based on - https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class MyResNet(nn.Module):
    # Based on PyTorch ResNet-18

    def __init__(self, block, layers, num_classes=10, zero_init_residual=False):
        super(MyResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(512 * block.expansion, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(256, num_classes),
        )

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
#         import pdb; pdb.set_trace()
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return F.log_softmax(x, dim=1)


In [ ]:
learn = Learner(databunch, MyResNet(BasicBlock, [2, 2, 2, 2]), metrics=accuracy)
learn.load('resnet_model_with_norm')
print('Model was loaded')

Model was loaded


In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.037798,0.115575,0.969600


## Ensemble of VGG and ResNet-18

In [ ]:
class VGG_ResNet(nn.Module):
    def __init__(self):
        super(VGG_ResNet, self).__init__()
        self.vgg = VGG()
        self.resnet = MyResNet(BasicBlock, [2, 2, 2, 2])

    def forward(self, x):
        vgg_out = self.vgg(x)
        resnet_out = self.resnet(x)
        out = (vgg_out + resnet_out) / 2
        return out

def vgg_resnet_load_model(learner, vgg_name, resnet_name):
        device = learner.data.device
        vgg_state = torch.load(learner.path/learner.model_dir/f'{vgg_name}.pth', map_location=device)
        learner.model.vgg.load_state_dict(vgg_state['model'], strict=True)

        resnet_state = torch.load(learner.path/learner.model_dir/f'{resnet_name}.pth', map_location=device)
        learner.model.resnet.load_state_dict(resnet_state['model'], strict=True)

In [ ]:
learn = Learner(databunch, VGG_ResNet(), metrics=accuracy)
# vgg_resnet_load_model(learn, vgg_name, resnet_name)
learn.load('vgg_resnet_model_with_norm')
print('Model was loaded')

Model was loaded


In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.015457,0.092744,0.989000


## Capsule Network

Taken from - https://github.com/higgsfield/Capsule-Network-Tutorial

In [ ]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=9):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=kernel_size,
                               stride=1
                             )

    def forward(self, x):
        return F.relu(self.conv(x))

class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=9):
        super(PrimaryCaps, self).__init__()

        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0)
                          for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0), 32 * 6 * 6, -1)
        return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

class DigitCaps(nn.Module):
    def __init__(self, num_capsules=10, num_routes=32 * 6 * 6, in_channels=8, out_channels=16):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)

        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))

        b_ij = b_ij.to(default_device)

        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = self.squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.reconstraction_layers = nn.Sequential(
            nn.Linear(16 * 10, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Sigmoid()
        )

    def forward(self, x, data):
        classes = torch.sqrt((x ** 2).sum(2))
        classes = F.softmax(classes)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.eye(10))

        masked = masked.to(default_device)
        masked = masked.index_select(dim=0, index=max_length_indices.squeeze(1).data)

        reconstructions = self.reconstraction_layers((x * masked[:, :, None, None]).view(x.size(0), -1))
        reconstructions = reconstructions.view(-1, 1, 28, 28)

        return reconstructions, masked


def caps_loss(inputs, targets):
    targets = torch.eye(10).index_select(dim=0, index=targets.cpu()).to(default_device)
    data, output, reconstructions, masked = inputs
    return margin_loss(output, targets) + reconstruction_loss(data, reconstructions)

def margin_loss(x, labels, size_average=True):
    batch_size = x.size(0)

    v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))
    left = F.relu(0.9 - v_c).view(batch_size, -1)
    right = F.relu(v_c - 0.1).view(batch_size, -1)

    loss = labels * left + 0.5 * (1.0 - labels) * right
    loss = loss.sum(dim=1).mean()

    return loss

def reconstruction_loss(data, reconstructions):
    loss = F.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
    return loss * 0.0005

def caps_accuracy(inputs, targs):
    masked = inputs[-1]
    predictions = np.argmax(masked.data.cpu().numpy(), 1)
    return torch.tensor((predictions == targs.cpu().numpy()).mean())


class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        self.conv_layer = ConvLayer()
        self.primary_capsules = PrimaryCaps()
        self.digit_capsules = DigitCaps()
        self.decoder = Decoder()

    def forward(self, data):
        output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))
        reconstructions, masked = self.decoder(output, data)
        return data, output, reconstructions, masked


In [ ]:
learn = Learner(databunch, CapsNet(), metrics=caps_accuracy, loss_func=caps_loss)
# vgg_resnet_load_model(learn, vgg_name, resnet_name)
learn.load('caps_net_model_with_norm')
print('Model was loaded')

Model was loaded


In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,caps_accuracy
1,0.006563,0.071374,0.976700


/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:88: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


## Ensemble of VGG and Capsule Network

Results are worse than VGG-ResNet Ensemble

In [ ]:
class VGG_Caps(nn.Module):
    def __init__(self):
        super(VGG_Caps, self).__init__()
        self.vgg = VGG()
        caps_model = CapsNet()
        self.capsnet = caps_model

    def forward(self, x):
        vgg_out = self.vgg(x)
        capsnet_out = self.capsnet(x)
        return vgg_out, capsnet_out


def vgg_capsnet_load_model(learner, vgg_name, caps_name):
        device = learner.data.device
        vgg_state = torch.load(learner.path/learner.model_dir/f'{vgg_name}.pth', map_location=device)
        learner.model.vgg.load_state_dict(vgg_state['model'], strict=True)

        capsnet_state = torch.load(learner.path/learner.model_dir/f'{caps_name}.pth', map_location=device)
        learner.model.capsnet.load_state_dict(capsnet_state['model'], strict=True)

def vgg_caps_accuracy(outputs, targs):
    caps_outputs = outputs[1][-1]
    vgg_outputs = outputs[0]
    batch_size = targs.size(0)

    caps_outputs = F.softmax(caps_outputs, dim=1)

    final_preds = (caps_outputs + vgg_outputs) / 2
    final_preds = final_preds.argmax(dim=-1).view(batch_size,-1)

    targs = targs.view(batch_size,-1)
    return (final_preds==targs).float().mean()

def vgg_caps_loss(inputs, targets):
    vgg_loss = torch.functional.F.nll_loss(inputs[0], targets)
    return caps_loss(inputs[1], targets) + vgg_loss

## Training

In [ ]:
learn = Learner(databunch, VGG_Caps(), metrics=vgg_caps_accuracy, loss_func=vgg_caps_loss)
vgg_capsnet_load_model(learn, 'vgg_model_with_norm', 'caps_net_model_with_norm')

In [ ]:
learn.fit(1)
# learn.save('vgg_resnet_model_with_norm')

epoch,train_loss,valid_loss,vgg_caps_accuracy
1,0.022337,0.150881,0.986300


/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:88: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
# Based on MNIST CNN from Keras' examples: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py (MIT License)

from __future__ import print_function
import keras
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import os
import random
import tensorflow as tf
import torch

# Set a common seed value
SEED_VALUE = 42

# 1. Set Python's general random seed
random.seed(SEED_VALUE)

# 2. Set NumPy's random seed
np.random.seed(SEED_VALUE)

# 3. Set TensorFlow's random seeds
# Set the global seed
tf.random.set_seed(SEED_VALUE)
# Set the seed for reproducible operations (if using older TensorFlow versions)
# tf.compat.v1.set_random_seed(SEED_VALUE) # Uncomment if needed for older TF

# 4. Set PyTorch's random seeds
# Note: Ensure PyTorch is imported if you are using it elsewhere
if 'torch' in globals() and torch.cuda.is_available():
    # Set CUDA seed for GPU operations
    torch.cuda.manual_seed_all(SEED_VALUE)
elif 'torch' in globals():
     # Set CPU seed
    torch.manual_seed(SEED_VALUE)


# 5. Set environment variables for TensorFlow/Keras and CuDNN reproducibility
# These might help with certain operations, though their effectiveness can vary
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # For newer TensorFlow
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' # For newer TensorFlow with CuDNN

# For older TensorFlow/CuDNN
# os.environ['TF_ENABLE_DETERMINISTIC_OPS'] = '1' # Uncomment if needed
# os.environ['CUDA_VISIBLE_DEVICES'] = '' # Uncomment to force CPU, helps with reproducibility

# --- END SUGGESTED CHANGE ---
batch_size = 128
num_classes = 10
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

def load(f):
    return np.load(f)['arr_0']

# Load the data
x_train = load('/content/kmnist-train-imgs.npz')
x_test = load('/content/kmnist-test-imgs.npz')
y_train = load('/content/kmnist-train-labels.npz')
y_test = load('/content/kmnist-test-labels.npz')

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('{} train samples, {} test samples'.format(len(x_train), len(x_test)))

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# --- SUGGESTED CHANGE ---
# Import the LearningRateScheduler callback
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adadelta

# Define a learning rate schedule function
# This function takes the epoch index as input and returns the learning rate
def step_decay(epoch):
    initial_lrate = 1.0 # Adadelta default learning rate is 1.0
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * (drop ** np.floor((epoch) / epochs_drop))
    print(f"Epoch {epoch}: Learning Rate = {lrate}") # Optional: Print the learning rate for monitoring
    return lrate

# Instantiate the LearningRateScheduler callback
lrate_scheduler = LearningRateScheduler(step_decay)

# Instantiate the optimizer (Adadelta in this case) - learning rate will be set by the scheduler
optimizer = Adadelta(learning_rate=step_decay(0)) # Set initial learning rate

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer, # Use the optimizer instance
              metrics=['accuracy'])

# --- SUGGESTED CHANGE ---
# Import the EarlyStopping callback
from keras.callbacks import EarlyStopping

# Instantiate the EarlyStopping callback
# Monitor validation loss ('val_loss')
# Stop training if 'val_loss' does not improve for 10 consecutive epochs (patience=10)
# Consider an improvement if the change is at least 0.001 (min_delta=0.001)
# Restore the model weights from the epoch with the best 'val_loss' (restore_best_weights=True)
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               min_delta=0.001,
                               restore_best_weights=True,
                               verbose=1) # Add verbose=1 to see when early stopping triggers

# Pass the EarlyStopping callback to the fit method in a list
callbacks_list = [early_stopping]

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=callbacks_list) # Pass the list of callbacks
# --- END SUGGESTED CHANGE ---
train_score = model.evaluate(x_train, y_train, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

60000 train samples, 10000 test samples
Epoch 0: Learning Rate = 1.0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7525 - loss: 0.7769 - val_accuracy: 0.8777 - val_loss: 0.4095
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9433 - loss: 0.1893 - val_accuracy: 0.9151 - val_loss: 0.2926
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9598 - loss: 0.1328 - val_accuracy: 0.9255 - val_loss: 0.2649
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9675 - loss: 0.1069 - val_accuracy: 0.9246 - val_loss: 0.2690
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9724 - loss: 0.0915 - val_accuracy: 0.9367 - val_loss: 0.2341
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9756 - loss: 0.0792 - val_accuracy: 0.9413 - val_loss: 0.2304
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9777 - loss: 0.0732 - val_accuracy: 0.9451 - val_loss: 0.2220
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9807 - loss: 0.0648 - val_accuracy: 0.

In [ ]:
# kNN with neighbors=4 benchmark for Kuzushiji-MNIST
# Acheives 92.10% test accuracy

from sklearn.neighbors import KNeighborsClassifier
import numpy as np

def load(f):
    return np.load(f)['arr_0']

# Load the data
x_train = load('kmnist-train-imgs.npz')
x_test = load('kmnist-test-imgs.npz')
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')

# Flatten images
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

clf = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fitting', clf)
clf.fit(x_train, y_train)
print('Evaluating', clf)

test_score = clf.score(x_test, y_test)
print('Test accuracy:', test_score)

Fitting KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')
Evaluating KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')
Test accuracy: 0.921
